# Übung: Vehicle routing problem als mip

Wir betrachten hier das vehicle routing problem ausgehend von Abschnitt 5.2).

\begin{align}
\min \sum_{e\in E}c_ex_e & & (1)\\
\sum_{e\in\delta^+(0)}x_e\leq k && (2)\\
\sum_{e\in\delta^-(v)}x_e=1 & \quad\text{für alle }v\in C & (3)\\
\sum_{e\in\delta^-(v)}x_e=\sum_{e\in\delta^+(v)}x_e & \quad\text{für alle }v\in C_0 & (4) \\
y_u+d_v-y_v\leq (1-x_{(u,v)})2q & \quad\text{für alle }(u,v)\in E\setminus\delta^-(0)& (5)\\
0\leq y_v\leq q&\quad\text{für alle }v\in C_0& (6)\\
y_0=0 && (7)\\
x_e\in\{0,1\}&\quad\text{für alle }e\in E & (8)
\end{align}


In dieser Übung soll das MIP des VRP um weitere Bedingungen erweitert werden.





## Zufallsinstanz

Wir erzeugen eine Zufallsinstanz mit euklidischen Distanzen und machen ein paar nötige imports.

In [ ]:
# Installation des Pakets mip
# Wenn Sie _nicht_ in Google Colab arbeiten, müssen Sie eventuell manuell das Paket installieren 
# In dem Fall kommentieren Sie die nächste Zeile aus
!pip install mip

import mip
import random
import math

def rnd_instance(n=10,k=3,seed=42):
    random.seed(seed)
    positions=[(random.random(),random.random()) for _ in range(n+1)]
    positions[0]=(0.5,0.5)  ## wir setzen das Depot in die Mitte
    d=[random.randint(1,3) for _ in positions]
    return positions,d

n=10 # Anzahl Kunden
k=3  # Anzahl Lieferwagen
q=10 # Kapazität der Lieferwagen
positions,d=rnd_instance(n=n,k=k)

## Das Grundmodell des MIPs

Als Basis ist hier noch einmal die Implementierung des MIPs wie oben beschrieben. Zunächst brauchen wir ein paar Hilfsfunktionen

In [ ]:
C=range(1,n+1) # Menge der Kunden
C0=range(n+1)   # Menge Kunden+Depot, das Depot hat Nr 0
# die Kantenmenge
E=[(u,v) for u in range(n+1) for v in range(n+1) if u!=v]

# Hilfsfunktionen: Indizies der ein- bzw ausgehenden Kanten bei einer Ecke v
def ingoing_edges(v):
    return [i for i,e in enumerate(E) if e[1]==v]

def outgoing_edges(v):
    return [i for i,e in enumerate(E) if e[0]==v]

# Der Abstand in der Ebene
def L2_dist(p,q):
    px,py=p
    qx,qy=q
    return math.sqrt((px-qx)**2+(py-qy)**2)

# Bequemlichkeitsfunktion
def dist(e):
    """berechnet die Euklidische Länge der Kante e=(u,v)"""
    u,v=e
    return L2_dist(positions[u],positions[v])

...und hier nun das MIP wie oben.

In [ ]:
m=mip.Model()

# die Variablen
x=[m.add_var(var_type=mip.BINARY) for _ in E]
y=[m.add_var(lb=0,ub=q) for _ in C0]
m+=y[0]==0


# Zielfunktion
m.objective=mip.minimize(mip.xsum(dist(e)*x[i] for i,e in enumerate(E)))

### Nebenbedinungen

# höchstens k Lieferwagen starten am Depot (2)
m+=mip.xsum(x[i] for i in outgoing_edges(0))<=k

# Jeder Kunde muss bedient werden (3)
for v in C:
    m+=mip.xsum(x[i] for i in ingoing_edges(v))==1

# Wenn ein Lieferwagen beim Kunden oder Depot anlangt, muss er dort auch wieder abfahren (4)
for v in C0:
    m+=mip.xsum(x[i] for i in ingoing_edges(v))-mip.xsum(x[i] for i in outgoing_edges(v))==0

# Jeder Lieferwagen, der einen Kunden anfährt, muss auch genügend Kapazität für den Kunden haben (5)
for i,e in enumerate(E):
    u,v=e
    if v!=0:  # schließe Kanten aus, die zum Depot gehen
        m+=y[u]+d[v]-y[v]<= (1-x[i])*2*q


### Aufgabe: Erweiterung
Wir wollen nun weitere Nebenbedingungen aus Abschnitt 5.3) einbauen.


1.   Für jeden benutzten LKW müssen Fixkosten in Höhe von $f$ bezahlt werden. LKWs, die unbenutzt im Lager bleiben kosten nichts. 
2.   Aus gesetzlichen Gründen darf ein LKW-Fahrer maximal $T$ Kilometer fahren. Die Strecke entlang einer Kante $e$ in Kilometern entspricht gerade den Kosten $c_e$.
2.   LKW-Fahrer, die besonders lange Routen fahren, erhalten Bonuszahlungen, die zusätzliche Kosten verursachen. Falls ein Fahrer mehr als $G$ km fahren muss, erhält er pro zusätzlich gefahrenen Kilometer $b$ €. 


Fügen Sie neue Variablen und Bedingungen ein und verändern Sie die Zielfunktion, sodass 1., 2. und 3. beachtet werden. Lösen Sie die Zufallsinstanz, die oben erzeugt wird. Damit die Lösung nicht ewig dauert, setzen Sie ein Zeitlimit, und zwar so: <code>m.optimize(max_seconds=100)</code>. Damit erhalten wir unter Umständen nicht die optimale Lösung, aber andererseits müssen wir höchstens 100 Sekunden warten. Geben Sie die Kosten (Wert der Zielfunktion) aus.

*Tipp für Teil 2 und 3: Fügen Sie für jeden Kunden $v$ eine Variable $s_v$ ein, die die Strecke messen soll, die der LKW auf seiner Tour bis zu $v$ benötigt.*


In [ ]:
# Die neuen Eingaben
f = 3 # Fixkosten pro genutztem LKW
T = 3 # maximale Routenlänge 
G = 0.5 # Routenlänge ab der Bonuszahlungen fällig werden
b = 1 # Zusatzkosten ab Routenlänge G

In [ ]:
### Ihre Code hier ### 